In [1]:
import PySpin
import cv2
import numpy as np
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
import datetime
import csv

import glob
import pickle

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def configure_camera(camera):
    # Changes amount of frame captured in camera
    width = int(2448 / 1)
    height = int(2048 / 1)

    # Reset Camera
    nodemap = camera.GetNodeMap()
    user_set_selector = PySpin.CEnumerationPtr(nodemap.GetNode("UserSetSelector"))
    default_set = user_set_selector.GetEntryByName("Default")
    user_set_selector.SetIntValue(default_set.GetValue())
    user_set_load = PySpin.CCommandPtr(nodemap.GetNode("UserSetLoad"))
    user_set_load.Execute()
    
    if camera.PixelFormat.GetAccessMode() == PySpin.RW:
        camera.PixelFormat.SetValue(PySpin.PixelFormat_RGB8)
        print("Pixel format set to color.")
    else:
        print("Color pixel format not supported.")

    # Retrieve the nodemap
    nodemap = camera.GetNodeMap()

    # Get width and height nodes
    node_width = PySpin.CIntegerPtr(nodemap.GetNode("Width"))
    node_height = PySpin.CIntegerPtr(nodemap.GetNode("Height"))
    node_width_max = node_width.GetMax()
    node_height_max = node_height.GetMax()

    # Calculate offsets to center the image
    offset_x = (node_width_max - width) // 2
    offset_y = (node_height_max - height) // 2

    # Set width and height
    node_offset_x = PySpin.CIntegerPtr(nodemap.GetNode("OffsetX"))
    node_offset_y = PySpin.CIntegerPtr(nodemap.GetNode("OffsetY"))

    node_width.SetValue(width)
    node_height.SetValue(height)
    node_offset_x.SetValue(offset_x)
    node_offset_y.SetValue(offset_y)

    camera.AcquisitionMode.SetValue(PySpin.AcquisitionMode_SingleFrame)

In [4]:
def capture_and_process_image(camera, model, video_writer, csv_writer):
    global mtx, dist, newcameramtx, roi

    #Capture Image
    camera.BeginAcquisition()
    image_result = camera.GetNextImage()
    if image_result.IsIncomplete():
        print('Image incomplete with image status %d ...' % image_result.GetImageStatus())
        image_result.Release()
        camera.EndAcquisition()
        return True

    image_data = image_result.GetNDArray()
    image_result.Release()
    camera.EndAcquisition()

    # Run lens calibration
    image_data = cv2.undistort(image_data, mtx, dist, None, newcameramtx)

    x, y, w, h = roi
    image_data = image_data[y:y+h, x:x+w]
    
    # Convert to a format that can be displayed with OpenCV
    image_data = cv2.resize(image_data, (640, 640))
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    
    # Perform detection
    results = model.predict(image_data, classes=0, agnostic_nms=False, verbose=False)[0]
    annotator = Annotator(image_data)
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    for r in results:
        for box in r.boxes:
            annotator.box_label(box.xyxy[0], model.names[int(box.cls)])
            csv_writer.writerow([timestamp, *box.xyxy[0], box.conf[0], model.names[int(box.cls)]])

    # Save the frame to video
    video_writer.write(image_data)

    # Display the annotated image
    annotated_image = annotator.result()
    cv2.imshow('Processed Image', annotated_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        return False

    return True



In [5]:
def main():
    global mtx, dist, newcameramtx, roi
    # Load in variables from lens calibration
    with open('calibration_data.pkl', 'rb') as f:
        data = pickle.load(f)
        
    mtx = data['mtx']
    dist = data['dist']
    newcameramtx = data['newcameramtx']
    roi = data['roi']

    # Load up camera
    system = PySpin.System.GetInstance()
    cam_list = system.GetCameras()
    if cam_list.GetSize() == 0:
        print('No cameras found.')
        return

    camera = cam_list.GetByIndex(0)
    camera.Init()
    configure_camera(camera)

    # Load your YOLO model
    model = YOLO("yolov8n.pt")  
    
    # Setup video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (640, 640))

    # Setup CSV writer
    csv_file = open('detections.csv', 'w', newline='')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Timestamp', 'X1', 'Y1', 'X2', 'Y2', 'Confidence', 'Class'])

    
    # Loop image capture
    while True:
        if not capture_and_process_image(camera, model, video_writer, csv_writer):
            break

    camera.DeInit()
    del camera
    cam_list.Clear()
    system.ReleaseInstance()
    cv2.destroyAllWindows()
    video_writer.release()
    csv_file.close()

    %run extractData
    %run sendData

In [ ]:
if __name__ == '__main__':
    main()

Pixel format set to color.


qt.qpa.xcb: QXcbConnection: XCB error: 148 (Unknown), sequence: 186, resource id: 0, major code: 140 (Unknown), minor code: 20
